
# Classification

The following example shows how to fit a simple classification model with
*auto-sklearn*.


In [1]:
from pprint import pprint
import sklearn.metrics
import autosklearn.classification
import pandas as pd
from sklearn.model_selection import train_test_split
import autosklearn

## Data Loading



In [6]:

file_path = '../data/titanic_dirty_data.csv'
# file_path = '../data/airbnb.csv'


df = pd.read_csv(file_path)
print(len(df))
df = df.dropna(subset=['Survived'])
y = df['Survived']
X = df.drop('Survived', axis=1)

# df = df.dropna(subset=['Rating'])
# y = df['Rating']
# X = df.drop('Rating', axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


624


## Build and fit a classifier



In [9]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=10,
    include = {
        'classifier': ["gradient_boosting"],
    },
    tmp_folder="tmp/autosklearn_classification_example_tmp5",
)

automl.fit(X_train, y_train)
run_key = list(automl.automl_.runhistory_.data.keys())[0]
run_value = automl.automl_.runhistory_.data[run_key]
config=automl.automl_.runhistory_.ids_config[run_key.config_id]
print(config)

/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/data/feature_validator.py:298: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(X[column]):
/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Name has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Sex has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Ticket has generic type object. Autosklearn will treat this column as string. Please 

[ERROR] [2023-12-05 19:27:58,822:Client-AutoML(1):4eb0a365-93ce-11ee-88ce-c33e0a0c4a01] The provided component 'gradient_boost' for the key 'classifier' in the 'include' argument is not valid. The supported components for the step 'classifier' for this task are ['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'mlp', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']
Traceback (most recent call last):
  File "/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/automl.py", line 761, in fit
    self.configuration_space, configspace_path = self._create_search_space(
  File "/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/automl.py", line 2263, in _create_search_space
    configuration_space = pipeline.get_configuration_space(
  File "/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/util/pipeline.py", line 53, in get_configu

Fitting to the training data: 100%|██████████| 120/120 [00:01<00:00, 119.75it/s, The total time budget for this task is 0:02:00]


ValueError: The provided component 'gradient_boost' for the key 'classifier' in the 'include' argument is not valid. The supported components for the step 'classifier' for this task are ['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'mlp', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']

## View the models found by auto-sklearn



In [4]:
print(automl.leaderboard())

          rank  ensemble_weight type      cost  duration
model_id                                                
4            1             0.12  mlp  0.115152  0.947709
48           2             0.04  mlp  0.121212  1.002826
37           3             0.08  mlp  0.145455  1.086634
57           4             0.04  mlp  0.151515  0.898793
13           5             0.06  mlp  0.157576  0.754961
14           6             0.02  mlp  0.169697  1.530150
20           7             0.04  mlp  0.169697  0.999433
9            8             0.08  mlp  0.175758  1.377708
44          10             0.08  mlp  0.175758  1.068163
62           9             0.02  mlp  0.175758  1.146705
28          11             0.06  mlp  0.181818  1.001655
33          12             0.10  mlp  0.181818  1.751342
5           14             0.10  mlp  0.224242  0.692738
54          13             0.02  mlp  0.224242  1.528497
55          15             0.02  mlp  0.260606  1.816128
29          16             0.02

## Print the final ensemble constructed by auto-sklearn



In [5]:
pprint(automl.show_models(), indent=4)

{   4: {   'balancing': Balancing(random_state=1, strategy='weighting'),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fa8d51f11c0>,
           'cost': 0.11515151515151512,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fa8d51e5b20>,
           'ensemble_weight': 0.12,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fa8d51f1d60>,
           'model_id': 4,
           'rank': 1,
           'sklearn_classifier': MLPClassifier(alpha=4.2841884333778574e-06, beta_1=0.999, beta_2=0.9,
              hidden_layer_sizes=(64, 64, 64),
              learning_rate_init=0.0011804284312897009, max_iter=128,
              n_iter_no_change=32, random_state=1, validation_fraction=0.0,
              verbose=0, warm_start=True)},
    5: {   'balancing': Balancing(random_state=1),
           'classi

## Get the Score of the final ensemble



In [8]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
print("Precision", sklearn.metrics.precision_score(y_test, predictions))
print("Recall", sklearn.metrics.recall_score(y_test, predictions))


Accuracy score: 0.736
Precision 0.7857142857142857
Recall 0.4489795918367347


/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_s

In [7]:

from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt

file_path = '../data/clean_titanic_data_rf.csv'
# file_path = '../data/clean_airbnb_data_gb.csv'


df = pd.read_csv(file_path)

# df = df.dropna(subset=['Survived'])
# y = df['Survived']
# X = df.drop('Survived', axis=1)

df = df.dropna(subset=['Rating'])
y = df['Rating']
X = df.drop('Rating', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

importances = forest.feature_importances_
print(len(importances))
sorted_indices = np.argsort(importances)[::-1]
print(sorted_indices)


106
[  6   7  14   5   9  74  12  91   1  13  81  76  10  96  93  79  57  51
  39  83  65  73  52  19  97  86  77  31  34  24  29  82   8  23  54  85
  27  64  11  87  44  25   2  59  20  90  88  56  41  99  80  48  21  15
  58 101  22  38  89  92  35  63  69  45  18  61  60   0  66  84  98  55
  37  50  47  36  40 100  70  53  68  17  43  32  30  49  71  46  94 102
  33  67  42  26  16 103  28  78  75 105  72  62  95 104   3   4]


/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


In [10]:
rf_predictions = forest.predict(X_test)
print(set(rf_predictions))
print(set(y_test))
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, rf_predictions))
print("Precision", sklearn.metrics.precision_score(y_test, rf_predictions))
print("Recall", sklearn.metrics.recall_score(y_test, rf_predictions))

{0.0, 1.0}
{0.0, 1.0}
Accuracy score: 0.848
Precision 0.7906976744186046
Recall 0.7727272727272727


/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
